In [12]:
%matplotlib notebook

from matplotlib import pylab
pylab.rcParams['figure.figsize'] = (10.0, 10.0)

from tensorflow.examples.tutorials.mnist import input_data

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

### Initialisations

Let us write some helper functions to initialise weights and biases. We'll initialise weights as Gaussian random variables with mean 0 and variance 0.0025. For biases we'll initialise everything with a constant 0.1. This is because we're mainly going to be using ReLU non-linearities.

In [13]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.05)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

### Model

Let's define the model. The model is defined as follows:

* An input that is 728 dimensional vector. 
* Reshape the input as 28x28x1 images (only 1 because they are grey scale) 
* A convolutional layer with 25 filters of shape 12x12x1 and a ReLU non-linearity (with stride (2, 2) and no padding)
* A convolutional layer with 64 filters of shape 5x5x25 and a ReLU non-linearity (with stride (1, 2) and padding to maintain size)
* A max_pooling layer of shape 2x2
* A fully connected layer taking all the outputs of the max_pooling layer to 1024 units and ReLU nonlinearity
* A fully connected layer taking 1024 units to 10 no activation function (the softmax non-linearity will be included in the loss function rather than in the model)

In [14]:
x = tf.placeholder(tf.float32, shape=[None, 784])
x_ = tf.reshape(x, [-1, 28, 28, 1])
y_ = tf.placeholder(tf.float32, shape=[None, 4])

# Define the first convolution layer here
W_conv1 = weight_variable([12, 12, 1, 25])
b_conv1 = bias_variable([25])
h_conv1 = tf.nn.relu(tf.nn.conv2d(x_, W_conv1, strides=[1, 2, 2, 1], padding='VALID') + b_conv1)

# Define the second convolution layer here
W_conv2 = weight_variable([5, 5, 25, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(tf.nn.conv2d(h_conv1, W_conv2,strides=[1, 1, 1, 1], padding='SAME') + b_conv2)

# Define maxpooling
h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

# All subsequent layers will be fully connected ignoring geometry so we'll flatten the layer
# Flatten the h_pool2_layer (as it has a multidimensiona shape) 
h_pool2_flat = tf.reshape(h_pool2, [-1, 5*5*64])
# Define the first fully connected layer here
W_fc1 = weight_variable([5* 5 * 64, 1024])
b_fc1 = bias_variable([1024])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Use dropout for this layer (should you wish)
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# The final fully connected layer - binary
W_fc2 = weight_variable([1024, 4])
b_fc2 = bias_variable([4])
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

#### Loss Function, Accuracy and Training Algorithm

* We'll use the cross entropy loss function. The loss function is called `tf.nn.cross_entropy_with_logits` in tensorflow

* Accuray is simply defined as the fraction of data correctly classified

* For training you should use the AdamOptimizer (read the documentation) and set the learning rate to be 1e-4. You are welcome, and in fact encouraged, to experiment with other optimisation procedures and learning rates. 

* (Optional): You may even want to use different filter sizes once you are finished with experimenting with what is asked in this practial

In [41]:
# We'll use the cross entropy loss function 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y_))

# And classification accuracy
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# And the Adam optimiser
train_step = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cross_entropy)

In [42]:
# Load the mnist data
mnist = input_data.read_data_sets('MNIST_data')
temp = [bin(i).lstrip('-0b').zfill(4) for i in mnist.train.labels]
train_labels = np.array([[int(j[0]),int(j[1]),int(j[2]),int(j[3])] for j in temp ])

temp = [bin(i).lstrip('-0b').zfill(4) for i in mnist.test.labels]
test_labels = np.array([[int(j[0]),int(j[1]),int(j[2]),int(j[3])] for j in temp ])


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [45]:
# Start a tf session and run the optimisation algorithm
sess = tf.Session()
sess.run(tf.global_variables_initializer())

with sess.as_default():
# sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    i = range(0,10000,100)
    j = range(100,11000,100)

    for idx in range(100):
#     for i in range(1000):  
        batchi = i[idx]
        batchj = j[idx]
        print (batchi)
        print (batchj)
        batch = [mnist.train.images[batchi:batchj], train_labels[batchi:batchj]]
        print (train_labels[batchi:batchj])
        print (mnist.train.labels[batchi:batchj])
        # Write the optimisation code here
        train_accuracy = accuracy.eval(feed_dict={
             x: batch[0], y_: batch[1], keep_prob: 1.0})
        print('step %d, training accuracy %g' % (idx, train_accuracy))

        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})   
    print ('Test accuracy: %g' % sess.run(accuracy, feed_dict={x: mnist.test.images, y_: test_labels, keep_prob: 1.0}))

0
100
[[0 1 1 1]
 [0 0 1 1]
 [0 1 0 0]
 [0 1 1 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 0 0]
 [1 0 0 1]
 [1 0 0 0]
 [0 0 0 0]
 [0 0 1 1]
 [0 0 0 1]
 [0 0 1 0]
 [0 1 1 1]
 [0 0 0 0]
 [0 0 1 0]
 [1 0 0 1]
 [0 1 1 0]
 [0 0 0 0]
 [0 0 0 1]
 [0 1 1 0]
 [0 1 1 1]
 [0 0 0 1]
 [1 0 0 1]
 [0 1 1 1]
 [0 1 1 0]
 [0 1 0 1]
 [0 1 0 1]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 1 1]
 [0 1 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [0 1 1 1]
 [0 0 1 1]
 [0 1 1 0]
 [0 1 0 0]
 [0 1 1 0]
 [0 1 1 0]
 [0 0 1 1]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 1]
 [1 0 0 1]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 0 0]
 [0 1 1 1]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 1 0 1]
 [0 1 1 1]
 [0 0 0 1]
 [0 1 1 1]
 [0 1 0 1]
 [0 1 0 1]
 [1 0 0 1]
 [1 0 0 1]
 [0 1 0 0]
 [0 0 1 0]
 [0 1 0 1]
 [0 0 1 1]
 [0 1 1 1]
 [0 1 0 0]
 [0 1 1 0]
 [0 1 1 0]
 [0 0 0 0]
 [0 0 0 1]
 [0 0 0 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 1 0 0]
 [1 0 0 0]
 [0 1 0 1]
 [0 0 1 1]
 [0 1 0 1]
 [0 0 0 0]
 [0 0 0 0]
 [0 1 1 0]
 [0 1 0 0]
 [0 0 1 1]
 [1 0 0 0]
 [0 0 1 1]
 [0 1 1 1]
 [0 

800
900
[[1 0 0 0]
 [0 0 0 0]
 [0 1 1 0]
 [1 0 0 1]
 [0 1 1 0]
 [0 1 0 0]
 [1 0 0 1]
 [0 1 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 1 1 0]
 [0 1 1 1]
 [0 0 1 1]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 1 0]
 [1 0 0 1]
 [0 1 1 1]
 [1 0 0 0]
 [0 0 1 1]
 [0 1 1 1]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 1 1 1]
 [0 1 1 0]
 [1 0 0 0]
 [0 1 0 1]
 [1 0 0 1]
 [0 0 0 0]
 [0 0 1 1]
 [0 1 1 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 1 0 0]
 [1 0 0 1]
 [0 1 1 0]
 [0 0 1 1]
 [0 1 0 1]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 0 0]
 [0 0 1 0]
 [0 1 1 0]
 [0 1 0 1]
 [0 1 1 0]
 [0 1 1 0]
 [0 0 0 1]
 [0 1 1 1]
 [0 0 1 1]
 [1 0 0 0]
 [0 1 0 0]
 [0 1 1 1]
 [0 1 0 1]
 [0 1 0 1]
 [0 0 1 1]
 [0 0 0 1]
 [1 0 0 1]
 [0 1 1 0]
 [0 0 0 1]
 [0 1 0 0]
 [0 0 0 1]
 [1 0 0 1]
 [0 1 0 1]
 [0 0 1 0]
 [1 0 0 1]
 [1 0 0 1]
 [0 0 1 1]
 [0 0 1 1]
 [0 1 0 1]
 [0 1 0 1]
 [1 0 0 1]
 [0 0 1 1]
 [0 0 0 1]
 [0 1 1 0]
 [1 0 0 1]
 [0 1 1 0]
 [0 1 0 0]
 [0 0 0 0]
 [0 0 0 1]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 1 0]
 [0 0 0 0]
 [0 1 0 1]
 [0 0 0 0]
 [0 1 1 0]
 [

1600
[[0 0 1 1]
 [0 1 0 1]
 [0 0 0 0]
 [1 0 0 0]
 [0 1 0 1]
 [0 0 1 0]
 [1 0 0 0]
 [0 1 1 1]
 [1 0 0 0]
 [0 1 1 0]
 [0 1 1 1]
 [0 0 0 1]
 [1 0 0 0]
 [0 1 0 1]
 [1 0 0 1]
 [0 0 1 0]
 [0 0 1 1]
 [0 1 1 0]
 [1 0 0 1]
 [0 1 0 0]
 [1 0 0 1]
 [0 0 0 1]
 [0 0 1 1]
 [1 0 0 0]
 [0 1 0 1]
 [0 1 1 1]
 [0 1 1 0]
 [0 1 0 1]
 [0 1 1 1]
 [0 1 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 1]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 0 0]
 [0 0 1 1]
 [0 0 1 0]
 [0 1 1 0]
 [1 0 0 1]
 [0 0 1 1]
 [1 0 0 0]
 [0 0 1 1]
 [0 1 0 0]
 [0 0 1 0]
 [0 1 1 0]
 [0 0 1 0]
 [0 1 1 1]
 [0 1 1 0]
 [0 1 0 0]
 [0 1 1 0]
 [0 0 0 1]
 [0 1 0 1]
 [0 1 0 0]
 [0 1 0 1]
 [0 0 0 1]
 [0 0 1 1]
 [0 1 0 0]
 [0 0 1 1]
 [0 1 1 0]
 [1 0 0 0]
 [1 0 0 1]
 [0 0 0 1]
 [0 1 1 1]
 [0 1 1 0]
 [0 0 0 1]
 [0 1 0 1]
 [0 0 0 1]
 [0 1 1 0]
 [0 1 1 0]
 [1 0 0 0]
 [0 1 1 0]
 [0 0 0 1]
 [0 0 1 1]
 [1 0 0 1]
 [0 1 1 0]
 [0 1 1 1]
 [1 0 0 0]
 [0 1 1 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 1 1]
 [0 0 1 1]
 [0 0 0 1]
 [0 1 1 1]
 [1 0 0 1]
 [1 0 0 1]
 [0 1

step 22, training accuracy 0.31
2300
2400
[[1 0 0 0]
 [0 1 1 1]
 [0 1 0 1]
 [0 0 0 1]
 [0 1 1 1]
 [0 0 0 0]
 [0 1 1 1]
 [1 0 0 0]
 [1 0 0 0]
 [0 1 1 1]
 [1 0 0 1]
 [0 1 0 1]
 [1 0 0 1]
 [0 0 1 0]
 [0 1 1 1]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [0 1 0 1]
 [0 0 1 1]
 [0 1 0 1]
 [0 0 0 1]
 [0 0 0 0]
 [0 0 1 0]
 [0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 1 1 1]
 [0 0 1 1]
 [0 1 0 0]
 [0 1 1 1]
 [1 0 0 1]
 [0 0 1 1]
 [0 0 1 0]
 [0 0 0 0]
 [0 0 1 1]
 [0 1 1 1]
 [0 1 1 0]
 [0 1 1 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 1 0 1]
 [0 0 1 0]
 [0 1 1 1]
 [0 0 0 1]
 [0 1 0 0]
 [0 1 0 1]
 [0 1 1 1]
 [0 0 1 0]
 [0 1 0 1]
 [0 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 0 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 1 1 0]
 [0 1 1 1]
 [0 0 0 1]
 [0 0 0 0]
 [1 0 0 1]
 [0 1 0 0]
 [0 0 1 1]
 [0 1 0 1]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 1 0 1]
 [0 0 1 0]
 [1 0 0 0]
 [0 1 0 1]
 [0 0 0 0]
 [0 1 1 0]
 [0 1 1 1]
 [1 0 0 1]
 [0 1 0 1]
 [0 0 1 1]
 [0 1 1 1]
 [0 1 1 0]
 [0 1 1 1]
 

3100
3200
[[0 1 1 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 1 0 1]
 [0 1 0 1]
 [0 1 1 0]
 [0 0 0 1]
 [0 1 1 1]
 [0 0 0 0]
 [1 0 0 0]
 [0 0 0 0]
 [1 0 0 1]
 [0 0 0 1]
 [0 0 0 0]
 [0 1 1 1]
 [0 0 1 1]
 [0 0 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [0 1 0 1]
 [0 0 1 1]
 [0 1 1 0]
 [0 0 1 1]
 [0 1 1 1]
 [0 0 0 1]
 [1 0 0 0]
 [0 1 0 1]
 [1 0 0 1]
 [0 1 0 1]
 [0 0 0 0]
 [0 1 1 1]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 1 0]
 [0 0 0 0]
 [0 0 1 1]
 [1 0 0 0]
 [0 1 1 1]
 [0 1 1 1]
 [1 0 0 0]
 [0 1 1 0]
 [1 0 0 1]
 [0 1 1 1]
 [0 1 1 1]
 [0 1 1 1]
 [0 1 1 0]
 [0 1 1 0]
 [0 0 0 0]
 [1 0 0 0]
 [0 1 1 0]
 [0 1 1 0]
 [0 1 1 0]
 [0 0 1 0]
 [0 1 1 0]
 [0 0 1 0]
 [1 0 0 1]
 [0 0 0 1]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 1 1]
 [0 0 1 1]
 [1 0 0 1]
 [0 0 0 1]
 [0 1 1 1]
 [0 1 1 1]
 [0 0 1 1]
 [0 1 1 1]
 [1 0 0 0]
 [0 0 0 0]
 [0 1 0 1]
 [0 1 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 1 0 1]
 [0 1 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 1 1 1]
 [0 1 0 1]
 [0 0 1 1]
 [0 0 0 1]
 [0 1 1 1]
 [0 0 0 0]
 [0 0 1 1]
 [0 0 0 1]


...

6400
[[1 0 0 1]
 [0 0 1 1]
 [1 0 0 0]
 [0 1 0 0]
 [0 0 0 0]
 [0 1 0 1]
 [0 0 1 0]
 [0 1 1 0]
 [0 0 0 1]
 [0 1 1 1]
 [0 1 0 1]
 [1 0 0 0]
 [0 0 0 1]
 [1 0 0 1]
 [0 1 0 0]
 [0 0 0 0]
 [0 1 1 0]
 [0 0 0 1]
 [0 0 0 0]
 [0 0 1 0]
 [0 0 0 0]
 [0 0 1 1]
 [1 0 0 0]
 [0 1 0 0]
 [0 1 0 1]
 [0 1 0 1]
 [0 0 0 1]
 [0 1 1 0]
 [0 1 1 1]
 [0 1 1 1]
 [0 0 0 0]
 [1 0 0 0]
 [0 1 1 1]
 [1 0 0 1]
 [0 1 1 1]
 [0 0 0 0]
 [0 1 1 0]
 [0 0 0 1]
 [0 1 0 1]
 [0 0 1 0]
 [0 0 0 0]
 [0 0 1 1]
 [0 0 0 1]
 [0 1 0 0]
 [0 0 1 0]
 [0 1 0 1]
 [0 0 1 0]
 [0 1 1 0]
 [0 1 1 0]
 [0 1 1 1]
 [0 1 1 1]
 [1 0 0 0]
 [0 0 0 1]
 [1 0 0 1]
 [0 0 0 0]
 [1 0 0 0]
 [0 1 0 1]
 [0 1 1 0]
 [0 0 0 1]
 [0 1 0 1]
 [0 0 0 1]
 [0 0 0 0]
 [0 0 0 1]
 [0 1 1 0]
 [0 0 1 0]
 [1 0 0 0]
 [1 0 0 1]
 [1 0 0 1]
 [1 0 0 1]
 [0 1 0 0]
 [0 1 1 1]
 [0 0 0 1]
 [0 1 1 1]
 [1 0 0 1]
 [1 0 0 0]
 [0 1 0 1]
 [1 0 0 1]
 [0 0 1 1]
 [0 1 0 0]
 [0 0 0 0]
 [0 0 1 0]
 [0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [1 0 0 1]
 [0 1 1 0]
 [0 0 0 1]
 [1 0 0 1]
 [0 1 0 0]
 [0 1

7100
7200
[[0 0 1 1]
 [0 1 1 0]
 [0 1 0 0]
 [0 0 0 0]
 [0 1 1 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 0 0]
 [0 0 1 1]
 [0 0 1 1]
 [0 1 0 1]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 1 1 0]
 [0 0 0 0]
 [0 1 1 1]
 [0 0 1 1]
 [1 0 0 0]
 [0 0 0 0]
 [1 0 0 1]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 1 1]
 [0 0 1 1]
 [0 1 0 0]
 [0 1 1 1]
 [1 0 0 1]
 [0 0 0 1]
 [0 0 1 0]
 [1 0 0 0]
 [1 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 1 0 1]
 [0 1 1 1]
 [0 1 1 1]
 [0 0 0 0]
 [0 0 0 1]
 [0 0 0 0]
 [0 1 1 1]
 [0 0 0 0]
 [0 0 0 1]
 [0 1 0 1]
 [0 1 1 0]
 [0 0 1 1]
 [0 1 1 1]
 [0 0 0 0]
 [0 0 1 1]
 [0 0 1 1]
 [0 0 1 1]
 [0 0 0 1]
 [0 1 1 0]
 [0 0 1 0]
 [0 1 0 1]
 [0 0 0 1]
 [0 1 1 0]
 [1 0 0 1]
 [0 0 1 0]
 [1 0 0 1]
 [1 0 0 0]
 [1 0 0 0]
 [0 1 1 1]
 [0 0 0 1]
 [0 1 1 0]
 [0 0 0 0]
 [0 1 0 0]
 [0 1 1 0]
 [1 0 0 1]
 [1 0 0 1]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 1]
 [0 1 1 1]
 [0 1 1 0]
 [0 0 0 1]
 [1 0 0 1]
 [0 0 0 0]
 [0 1 0 1]
 [1 0 0 0]
 [0 0 1 1]
 [0 0 0 0]
 [0 1 1 1]
 [0 0 1 1]
 [0 1 0 0]
 [0 1 1 1]
 [0 0 1 1]


7900
8000
[[1 0 0 1]
 [0 0 1 1]
 [0 1 1 0]
 [1 0 0 1]
 [1 0 0 0]
 [0 1 1 0]
 [0 0 1 0]
 [0 0 0 0]
 [0 1 1 1]
 [0 0 0 1]
 [1 0 0 1]
 [0 0 1 0]
 [0 1 0 0]
 [0 0 1 1]
 [1 0 0 1]
 [0 1 0 0]
 [0 0 1 0]
 [0 1 0 1]
 [0 0 0 1]
 [0 1 1 0]
 [0 0 1 1]
 [0 1 1 1]
 [0 1 0 0]
 [1 0 0 0]
 [0 0 1 1]
 [1 0 0 1]
 [0 0 0 0]
 [0 0 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 1 1]
 [1 0 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [0 1 0 1]
 [0 0 1 0]
 [0 1 1 0]
 [0 1 0 1]
 [0 1 1 1]
 [0 0 1 1]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 1]
 [1 0 0 1]
 [0 0 0 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 1 1]
 [0 0 1 0]
 [0 1 0 0]
 [0 1 1 1]
 [0 1 0 1]
 [1 0 0 0]
 [0 1 1 0]
 [1 0 0 0]
 [0 1 1 1]
 [0 1 0 1]
 [1 0 0 0]
 [0 1 1 0]
 [1 0 0 1]
 [0 1 0 0]
 [0 0 1 1]
 [0 0 1 1]
 [0 1 0 0]
 [0 1 1 1]
 [0 0 0 1]
 [0 1 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [0 1 1 0]
 [0 1 0 0]
 [0 1 1 0]
 [0 1 0 1]
 [0 1 1 1]
 [0 1 1 0]
 [0 0 1 1]
 [0 1 1 1]
 [0 0 1 1]
 [1 0 0 0]
 [0 0 1 0]
 [0 1 0 1]
 [0 0 0 1]
 [1 0 0 0]
 [1 0 0 1]


step 87, training accuracy 0.11
8800
8900
[[0 1 0 1]
 [1 0 0 1]
 [0 1 1 1]
 [1 0 0 0]
 [0 0 1 0]
 [1 0 0 1]
 [1 0 0 1]
 [0 0 0 0]
 [0 0 0 0]
 [0 1 1 0]
 [0 1 1 1]
 [0 1 0 1]
 [1 0 0 0]
 [0 1 1 0]
 [0 1 1 1]
 [0 1 1 1]
 [0 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 1 1 1]
 [0 0 1 0]
 [0 1 0 1]
 [0 0 0 0]
 [0 1 0 0]
 [0 0 1 1]
 [0 0 0 1]
 [0 0 0 0]
 [0 0 1 1]
 [0 1 1 0]
 [0 1 0 1]
 [0 0 0 0]
 [0 0 1 1]
 [0 1 1 0]
 [0 0 0 0]
 [0 1 1 1]
 [0 0 0 1]
 [0 0 1 1]
 [0 0 1 0]
 [0 1 1 0]
 [0 0 1 1]
 [0 0 0 1]
 [0 1 0 0]
 [0 1 1 1]
 [0 1 0 1]
 [0 0 1 0]
 [0 1 1 0]
 [0 1 1 0]
 [0 1 1 1]
 [0 1 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [1 0 0 1]
 [0 1 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 1]
 [0 1 0 0]
 [0 0 1 0]
 [0 1 1 0]
 [0 0 1 1]
 [0 1 1 0]
 [0 1 0 0]
 [0 1 0 1]
 [0 1 0 1]
 [0 1 0 0]
 [0 1 1 0]
 [1 0 0 0]
 [0 1 1 1]
 [0 1 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [1 0 0 1]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 1 0]
 [0 1 0 0]
 [0 0 1 1]
 [0 0 0 0]
 [0 1 1 0]
 [0 0 0 1]
 [0 1 1 1]
 [0 0 0 1]
 [1 0 0 0]
 [0 1 0 1]
 

9600
9700
[[0 1 1 1]
 [0 1 1 1]
 [0 1 0 0]
 [0 0 1 0]
 [0 1 1 0]
 [0 1 1 1]
 [0 0 0 1]
 [1 0 0 1]
 [0 1 1 0]
 [0 1 0 1]
 [0 1 1 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 1 0 0]
 [0 0 1 0]
 [0 1 1 0]
 [0 0 1 1]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 1 1]
 [0 1 0 1]
 [0 0 1 1]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 1]
 [0 1 0 1]
 [0 0 0 0]
 [0 1 0 1]
 [0 0 0 1]
 [0 1 1 0]
 [0 0 1 0]
 [0 1 1 0]
 [0 0 1 1]
 [0 0 1 1]
 [0 1 0 0]
 [1 0 0 0]
 [0 1 0 1]
 [1 0 0 0]
 [0 1 1 0]
 [0 1 1 1]
 [0 1 1 1]
 [0 0 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [1 0 0 1]
 [1 0 0 0]
 [0 0 0 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 1 0 0]
 [0 0 1 0]
 [0 1 0 0]
 [0 0 1 1]
 [0 0 0 1]
 [0 1 0 0]
 [0 1 1 0]
 [0 1 0 1]
 [0 1 1 0]
 [0 1 1 0]
 [1 0 0 1]
 [0 1 1 1]
 [1 0 0 0]
 [1 0 0 0]
 [0 1 1 0]
 [1 0 0 1]
 [0 0 0 0]
 [1 0 0 0]
 [0 1 0 1]
 [0 1 1 0]
 [0 1 0 1]
 [0 1 0 1]
 [0 0 1 1]
 [0 0 0 0]
 [0 1 1 0]
 [0 1 1 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 1]
 [1 0 0 1]
 [0 0 0 1]
 [0 1 0 0]
 [0 1 1 1]
 [0 0 0 1]
 [0 0 1 1]
 [1 0 0 1]
 [0 1 0 1]
 [0 1 0 1]


In [20]:
# Print accuracy on the test set
print ('Test accuracy: %g' % sess.run(accuracy, feed_dict={x: mnist.test.images, y_: test_labels, keep_prob: 1.0}))

Test accuracy: 0.2693


#### Visualising the Filters

We'll now visualise all the 32 filters in the first convolution layer. As they are each of shape 12x12x1, they may themselves be viewed as greyscale images. Visualising filters in further layers is more complicated and involves modifying the neural network. See the [paper](http://www.matthewzeiler.com/pubs/arxive2013/arxive2013.pdf) by Matt Zeiler and Rob Fergus if you are interested. 

In [ ]:
# Visualise the filters in the first convolutional layer
with sess.as_default():
    W = W_conv1.eval()
# Add code to visualise filters here
    fig = plt.figure(figsize=(5,5))
    for i in range(25):
        ax = fig.add_subplot(5, 5, i + 1)
        ax.set_xticks(())
        ax.set_yticks(())
        ax.imshow(W[:,:,0,i].reshape(12, 12), cmap='Greys_r')  



#### Identifying image patches that activate the filters

For this part you'll find the 12 patches in the test-set that activate each of the first 5 filters that maximise the activation for that filter.

In [ ]:
H =  sess.run(h_conv1, feed_dict={x: mnist.test.images})
# firstFive = H[:,:,:,0]
print (H.shape)

for j in range(5):
    feature = H[:,:,:,j]
    fig = plt.figure(figsize=(3,4))
    for i in range(12):
        imgIdx,row,column = np.unravel_index(feature.argmax(), feature.shape)
        img = mnist.test.images[imgIdx].reshape(28,28)[2*row:2*row+12,2*column:2*column+12]    
        feature[imgIdx,row,column] = 0.
        ax = fig.add_subplot(4,3,i + 1)
        ax.set_xticks(())
        ax.set_yticks(())
        ax.imshow(img, cmap='Greys_r')  

# Add code to visualise patches in the test set that find the most result in 
# the highest activations for filters 0, ... 4


### Model 2 Different Filter Size

Let's define the model. The model is defined as follows:

* An input that is 728 dimensional vector. 
* Reshape the input as 28x28x1 images (only 1 because they are grey scale) 
* A convolutional layer with 25 filters of shape 6x6x1 (instead of 12x12x1 ) and a ReLU non-linearity (with stride (2, 2) and no padding)
* A convolutional layer with 64 filters of shape 5x5x25 and a ReLU non-linearity (with stride (1, 2) and padding to maintain size)
* A max_pooling layer of shape 2x2
* A fully connected layer taking all the outputs of the max_pooling layer to 1024 units and ReLU nonlinearity
* A fully connected layer taking 1024 units to 10 no activation function (the softmax non-linearity will be included in the loss function rather than in the model)

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 784])
x_ = tf.reshape(x, [-1, 28, 28, 1])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

# Define the first convolution layer here
W_conv1 = weight_variable([6, 6, 1, 25])
b_conv1 = bias_variable([25])
h_conv1 = tf.nn.relu(tf.nn.conv2d(x_, W_conv1, strides=[1, 2, 2, 1], padding='VALID') + b_conv1)

# Define the second convolution layer here
W_conv2 = weight_variable([5, 5, 25, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(tf.nn.conv2d(h_conv1, W_conv2,strides=[1, 1, 1, 1], padding='SAME') + b_conv2)

# Define maxpooling
h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

# All subsequent layers will be fully connected ignoring geometry so we'll flatten the layer
# Flatten the h_pool2_layer (as it has a multidimensiona shape) 
print (h_pool2)
h_pool2_flat = tf.reshape(h_pool2, [-1, 6*6*64])
# Define the first fully connected layer here
W_fc1 = weight_variable([6* 6 * 64, 1024])
b_fc1 = bias_variable([1024])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Use dropout for this layer (should you wish)
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# The final fully connected layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


In [ ]:
# We'll use the cross entropy loss function 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y_))

# And classification accuracy
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# And the Adam optimiser
train_step = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cross_entropy)

In [ ]:
# Start a tf session and run the optimisation algorithm
sess2 = tf.Session()
sess2.run(tf.global_variables_initializer())

with sess2.as_default():
# sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    for i in range(4000):
#     for i in range(1000):        
        batch = mnist.train.next_batch(50)
        # Write the optimisation code here
        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict={
                 x: batch[0], y_: batch[1], keep_prob: 1.0})
            print('step %d, training accuracy %g' % (i, train_accuracy))

        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})   
    print ('Test accuracy: %g' % sess2.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

In [ ]:
# Print accuracy on the test set
print ('Test accuracy: %g' % sess2.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))